In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('cbs.db')

In [13]:
query = """
SELECT 
    x.AC_BRANCH,
    x.AC_NO,
    x.DRCR_IND,
    x.FINANCIAL_CYCLE,
    x.LCY_AMOUNT,
    x.PERIOD_CODE,
    x.PRODUCT,
    p.PRODUCT_DESCRIPTION AS PRODUCT_DESC,
    x.TRN_CODE,
    s.TRN_DESC AS TRN_DSC,
    x.TRN_DT,
    x.TRN_REF_NO,
    x.VALUE_DT
FROM 
    ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2025 x
LEFT JOIN 
    STTM_TRN_CODE s ON x.TRN_CODE = s.TRN_CODE
LEFT JOIN 
    CSTM_PRODUCT p ON x.PRODUCT = p.PRODUCT_CODE
WHERE 
    x.AC_NO IN ('AS120103', 'AS100601')
"""

# Run the query
df = pd.read_sql_query(query, conn)

# Reorder columns: insert TRN_DSC after TRN_CODE and PRODUCT_DESC after PRODUCT
columns = df.columns.tolist()

# Move PRODUCT_DESC after PRODUCT
if 'PRODUCT' in columns and 'PRODUCT_DESC' in columns:
    product_index = columns.index('PRODUCT')
    columns.insert(product_index + 1, columns.pop(columns.index('PRODUCT_DESC')))

# Move TRN_DSC after TRN_CODE
if 'TRN_CODE' in columns and 'TRN_DSC' in columns:
    trn_code_index = columns.index('TRN_CODE')
    columns.insert(trn_code_index + 1, columns.pop(columns.index('TRN_DSC')))

# Apply reordered columns
df = df[columns]

In [15]:
pd.set_option('display.max_columns', None)
df.head()

,AC_BRANCH,AC_CCY,AC_NO,AMOUNT_TAG,AUTH_ID,AUTH_STAT,BATCH_NO,DRCR_IND,EVENT,EXCH_RATE,FCY_AMOUNT,FINANCIAL_CYCLE,LCY_AMOUNT,MODULE,PRODUCT,PRODUCT_DESC,RELATED_ACCOUNT,RELATED_CUSTOMER,RELATED_REFERENCE,TRN_CODE,TRN_DSC,TRN_DT,TRN_REF_NO,TXN_INIT_DATE,USER_ID,VALUE_DT
0,999,TZS,AS100601,TFR_AMT,TIPS01,A,None,C,INIT,0.0,0.0,FY2025,10000.0,FT,SECW,SELCOM CASH WITHDRAWAL,None,14218257.0,None,S09,SELCOM AG CASH DEPOSIT,05/02/2025 00:00:00,001SECW251220544,05/02/2025 00:00:00,TIPS01,05/02/2025 00:00:00
1,999,TZS,AS100601,AMT_EQUIV,TIPS01,A,None,D,INIT,0.0,0.0,FY2025,430000.0,FT,SECD,SELCOM CASH DEPOSIT,None,1033176.0,None,S03,SELCOM AGENT TRANSFER,05/02/2025 00:00:00,001SECD251220038,05/02/2025 00:00:00,TIPS01,05/02/2025 00:00:00
2,999,TZS,AS100601,TFR_AMT,TIPS01,A,None,C,INIT,0.0,0.0,FY2025,160000.0,FT,SECW,SELCOM CASH WITHDRAWAL,None,340488.0,None,S09,SELCOM AG CASH DEPOSIT,05/02/2025 00:00:00,001SECW251220002,05/02/2025 00:00:00,TIPS01,05/02/2025 00:00:00
3,999,TZS,AS100601,AMT_EQUIV,TIPS01,A,None,D,INIT,0.0,0.0,FY2025,80925.0,FT,SECD,SELCOM CASH DEPOSIT,None,1033176.0,None,S03,SELCOM AGENT TRANSFER,05/02/2025 00:00:00,001SECD251220544,05/02/2025 00:00:00,TIPS01,05/02/2025 00:00:00
4,999,TZS,AS120103,TFR_AMT,ESB001,A,None,C,INIT,0.0,0.0,FY2025,2500.0,FT,DAPU,DIGITAL AGENT UTILITY PAYMENT,None,14203085.0,None,D29,DIGITAL AGENT UTILITY PAYMENT,05/02/2025 00:00:00,008DAPU251220001,05/02/2025 00:00:00,ESB001,05/01/2025 00:00:00


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172763 entries, 0 to 172762
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   AC_BRANCH          172763 non-null  int64  
 1   AC_CCY             172763 non-null  object 
 2   AC_NO              172763 non-null  object 
 3   AMOUNT_TAG         172763 non-null  object 
 4   AUTH_ID            172763 non-null  object 
 5   AUTH_STAT          172763 non-null  object 
 6   BATCH_NO           3081 non-null    object 
 7   DRCR_IND           172763 non-null  object 
 8   EVENT              172763 non-null  object 
 9   EXCH_RATE          172763 non-null  float64
 10  FCY_AMOUNT         172763 non-null  float64
 11  FINANCIAL_CYCLE    172763 non-null  object 
 12  LCY_AMOUNT         172763 non-null  float64
 13  MODULE             172763 non-null  object 
 14  PRODUCT            172763 non-null  object 
 15  PRODUCT_DESC       172395 non-null  object 
 16  RE

In [17]:
df.to_excel('TRANSACTIONS_AS100601_AS120103.xlsx', index=False)

## Fetching AS120160 from 2023 to 2025 April

In [6]:
query = """
SELECT 
    x.AC_BRANCH,
    x.AC_CCY,
    x.AC_NO,
    x.AMOUNT_TAG,
    x.AUTH_ID,
    x.AUTH_STAT,
    x.BATCH_NO,
    x.DRCR_IND,
    x.EVENT,
    x.EXCH_RATE,
    x.FCY_AMOUNT,
    x.FINANCIAL_CYCLE,
    x.LCY_AMOUNT,
    x.MODULE,
    x.PRODUCT,
    p.PRODUCT_DESCRIPTION AS PRODUCT_DESC,
    x.RELATED_ACCOUNT,
    x.RELATED_CUSTOMER,
    x.TRN_CODE,
    s.TRN_DESC AS TRN_DSC,
    x.TRN_DT,
    x.TRN_REF_NO,
    x.TXN_INIT_DATE,
    x.USER_ID,
    x.VALUE_DT
FROM 
    ACVWS_ALL_AC_ENTRIES_ACRJRNAL x
LEFT JOIN 
    STTM_TRN_CODE s ON x.TRN_CODE = s.TRN_CODE
LEFT JOIN 
    CSTM_PRODUCT p ON x.PRODUCT = p.PRODUCT_CODE
WHERE 
    x.AC_NO IN ('AS120160')
"""

# Run the query
df = pd.read_sql_query(query, conn)

# Reorder columns: insert TRN_DSC after TRN_CODE and PRODUCT_DESC after PRODUCT
columns = df.columns.tolist()

# Move PRODUCT_DESC after PRODUCT
if 'PRODUCT' in columns and 'PRODUCT_DESC' in columns:
    product_index = columns.index('PRODUCT')
    columns.insert(product_index + 1, columns.pop(columns.index('PRODUCT_DESC')))

# Move TRN_DSC after TRN_CODE
if 'TRN_CODE' in columns and 'TRN_DSC' in columns:
    trn_code_index = columns.index('TRN_CODE')
    columns.insert(trn_code_index + 1, columns.pop(columns.index('TRN_DSC')))

# Apply reordered columns
df = df[columns]

In [7]:
pd.set_option('display.max_columns', None)
df.head()

,AC_BRANCH,AC_CCY,AC_NO,AMOUNT_TAG,AUTH_ID,AUTH_STAT,BATCH_NO,DRCR_IND,EVENT,EXCH_RATE,FCY_AMOUNT,FINANCIAL_CYCLE,LCY_AMOUNT,MODULE,PRODUCT,PRODUCT_DESC,RELATED_ACCOUNT,RELATED_CUSTOMER,TRN_CODE,TRN_DSC,TRN_DT,TRN_REF_NO,TXN_INIT_DATE,USER_ID,VALUE_DT
0,29,TZS,AS120160,TFR_AMT,FINTECH,A,NaN,C,DRLQ,0.0,0.0,FY2024,900000.0,PC,IPPC,Incoming payment,029000008135,786080308.0,124,Incoming EFT payments,01/02/2024 00:00:00,029IPPC240020001,01/02/2024 00:00:00,FINTECH,01/02/2024 00:00:00
1,29,TZS,AS120160,TFR_AMT,FINTECH,A,NaN,D,CRLQ,0.0,0.0,FY2024,900000.0,PC,IPPC,Incoming payment,029000008135,786080308.0,124,Incoming EFT payments,01/02/2024 00:00:00,029IPPC240020001,01/02/2024 00:00:00,FINTECH,01/02/2024 00:00:00
2,25,TZS,AS120160,TFR_AMT,FCATOP,A,NaN,C,DRLQ,0.0,0.0,FY2024,1000000.0,PC,BKOP,Outgoing payment-Book trf,025000000017,315720.0,122,Outgoing EFT payment,01/02/2024 00:00:00,025BKOP240020001,01/02/2024 00:00:00,FCATOP,01/02/2024 00:00:00
3,29,TZS,AS120160,TFR_AMT,FINTECH,A,NaN,C,DRLQ,0.0,0.0,FY2024,3196882.0,PC,IPPC,Incoming payment,029000000144,157649308.0,124,Incoming EFT payments,01/02/2024 00:00:00,029IPPC2400200DX,01/02/2024 00:00:00,FINTECH,01/02/2024 00:00:00
4,29,TZS,AS120160,TFR_AMT,FINTECH,A,NaN,D,CRLQ,0.0,0.0,FY2024,3196882.0,PC,IPPC,Incoming payment,029000000144,157649308.0,124,Incoming EFT payments,01/02/2024 00:00:00,029IPPC2400200DX,01/02/2024 00:00:00,FINTECH,01/02/2024 00:00:00


In [8]:
df.to_excel('TRANSACTIONS_AS120160_2024.xlsx', index=False)

In [ ]:
conn.close()